In [ ]:
#
# Copyright (c) 2021 Be The Match.
#
# This file is part of BLEAT 
# (see https://github.com/nmdp-bioinformatics/b-leader).
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program. If not, see <http://www.gnu.org/licenses/>.
#
%load_ext jupyternotify
%load_ext autoreload
%autoreload 2

import requests
import json
import regex
import re
import time
import pandas as pd
import numpy as np
import operator
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

from bleader_seq.sequence import Sequence
from bleader_seq.hlab_map import HlaBMap
from bleader_seq.allele_list import AlleleList
from bleader_seq.subject import Subject
from bleader_seq.allele_family import AlleleFamily, Seq
from bleader_seq.leaders import Leaders
from bleader_seq.allele import Allele

## Leader Map

In [ ]:
leader_map = pd.read_csv("../data/table/leader_map.csv", 
                         names=['Allele', 'Leader'])
leader_map["Allele"] = "HLA-" + leader_map["Allele"]
leader_map["Leader"][leader_map["Leader"] == '-'] = 'M'
leader_map = leader_map.set_index('Allele').to_dict()['Leader']

In [ ]:
n_alleles = len(leader_map)
defined_n_alleles = len([leader for leader in leader_map.values() if leader != '*'])
coverage = defined_n_alleles / n_alleles * 100
print("Coverage is (%.2f%%) %s/%s" % (coverage, defined_n_alleles, n_alleles))

In [ ]:
len(leader_map)

# Subjects Import

Subjects were generated as a dictionary of ID's to Subject objects (containing sequences and metadata) in a separate, private notebook. They were pickled for loading here.

In [ ]:
subject_file = "data/cohort-1-011021.p"

Contains filters:
    - Only two leader peptides
    - PacBIO and NGS-WG only
    - QC IDs filtered out
    - RES projects filtered out
    - NMDP verified IDs only
    - Duplicate sequences (when there's heterozygous alleles)

In [ ]:
import pickle
with open(subject_file, "rb") as openfile:
    subjects = pickle.load(openfile)

In [ ]:
len(subjects)

There are 1,098,358 subjects to be loaded.

**Note:** Due to the large amount of data being loaded, this step will take almost an hour to complete.

# Cohort Characterization

## Alleles

In [ ]:
import sys
sys.path.insert(1, '../..')
from bleader.hla_b import HlaBAllotype

In [ ]:
len(subjects)

In [ ]:
len([allele_list for subject in subjects.values() for allele_list in subject.alleles.values()])

In [ ]:
all_alleles = []
num_diff_fields = 0
num_fields_set = []
num_split_alleles = 0
for subject in subjects.values():
    for allele_list in subject.alleles.values():
        num_fields = set()
        split_alleles = allele_list.allele_list.split('/')
        if len(split_alleles) > 1:
            num_split_alleles += 1
        for allele in split_alleles:
            all_alleles.append(allele)
            num_fields.add(len(allele.split(':')))
        if len(num_fields) > 1:
            num_diff_fields += 1
            print(allele_list.allele_list)
        else:
            num_fields_set.append(num_fields.pop())

In [ ]:
num_split_alleles

In [ ]:
pd.Series(num_fields_set).value_counts()

- 4    1590640
- 3     561613
- 2      44451

In [ ]:
num_diff_fields

12 allele lists have different numbers of fields

In [ ]:
len(all_alleles)

In [ ]:
unique_alleles = set(all_alleles)

In [ ]:
len(unique_alleles)

In [ ]:
xx_alleles = []
g_groups = []
for allele in unique_alleles:
    allele = HlaBAllotype(allele.replace('HLA-',''))
    if "XX" in allele.name:
        xx_alleles.append(allele)
    elif allele.g_group:
        g_groups.append(allele)
    else:
        if any([bool(re.match("[A-Z]+", field)) for field in allele.fields if field]):
            print(allele)
        else:
            if allele.resolution != 'high':
                print(allele)
                print(allele.__dict__)

In [ ]:
len(g_groups)

In [ ]:
len(xx_alleles)

- There is one MAC B\*39:BMFM
- 5,316,509 alleles within the list
- 2,259 unique alleles
- 44 G groups
- 30 XX alleles

### Project Names

In [ ]:
pd.Series([subject.project for subject in subjects.values()]).value_counts()

## Sequences

### Primers

In [ ]:
pd.Series([str(allele.whole_NTs[:18]) + '-' + str(allele.whole_NTs[-18:] + '-' + str(len(allele.whole_NTs)))
               for i, subject in enumerate(subjects.values())
               for allele in subject.alleles.values()]).value_counts()

- CACCCACCCGGACTCAGA-GTGTCTCTCACAGCTTGA-2712    482192
- CACCCACCCGGACTCAGA-GTGTCTCTCACAGCTTGA-2725    436819
- CACCCACCCGGACTCAGA-GTGTCTCTCACAGCTTGA-2711    308625
- CACCCACCCGGACTCAAA-GTGTCTCTCACAGCTTGA-2725    126373
- CACCCACCCGGACTCAGA-GTGTCTCTCACAGCTTGA-2713    101886
                                               ...  
- GGGCTCTCAGGGTCTCAG-TGGGGTCTAGAGTGGGCG-2962         1
- CACCCACCCGGACTCAGA-GTGTCTCTCACAGCTTGA-2685         1
- CAGGTCACGGGCTCTCAG-GAGGGACCGAGATGCAGC-3167         1
- GGGCTCTCAGTGTCTCAG-GGGTCTAGAGTGGGCGGG-2978         1
- GGGCTCTCAGGTCTCAGG-GAGGGACTGAGATGCAGC-3135         1
- Length: 1225, dtype: int64

### Lengths

In [ ]:
lengths = [len(allele.whole_NTs) for subject in subjects.values()
                         for allele in subject.alleles.values()]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(pd.Series(lengths).value_counts(normalize=True))

In [ ]:
from scipy import stats

In [ ]:
import statistics
print('- Average length is %.2f out of %s sequences' % (sum(lengths)/len(lengths), len(lengths)))
print('- The standard deviation is %.2f' % statistics.stdev(lengths))
print('- Minimimum is %s. Maximum is %s.' % (min(lengths), max(lengths)))
print('- Mode is %s.' % max(set(lengths), key=lengths.count))
q25, q50, q75 = np.percentile(lengths, [25, 50, 75])
print("- Q25: %s, Median: %s, Q75: %s" % (q25, q50, q75))

- Average length is 2851.34 out of 2196716 sequences
- The standard deviation is 449.27
- Minimimum is 1012. Maximum is 6836.
- Mode is 2725.
- Q25: 2712.0, Median: 2724.0, Q75: 2725.0

## Leader Peptides

In [ ]:
leader_peps = {}
exon1_seqs = {}
for subject in subjects.values():
    for allele in subject.alleles.values():
        leader_pep = allele.leader_peptide
        race = subject.broad_race
        if leader_pep not in leader_peps:
            leader_peps[leader_pep] = Leaders(leader_pep)
        leader_peps[leader_pep].add_leader(allele, race)
        
        exon1_seq = allele.exon1_NTs
        leader_NTs = allele.exon1_NTs[6:33]
        if Sequence('').translate(leader_NTs) != leader_pep:
            print("ERROR", subject)
            break
        if leader_NTs not in exon1_seqs:
            exon1_seqs[leader_NTs] = Seq(leader_NTs)
        exon1_seqs[leader_NTs].add_leader(allele, race)
total = sum([leaders.count for leaders in leader_peps.values()])
print("There are %s number of alleles" % total)

In [ ]:
leader_peps['VTAPRTLLL'].count
leaders = {}
for leader_seq, leader_pep in leader_peps.items():
    leader = leader_seq[1]
    if leader not in leaders:
        leaders[leader] = 0
    leaders[leader] += leader_pep.count
leaders
total_peps = sum(leaders.values())
for leader, count in leaders.items():
    print(leader, count, count / total_peps)

There are 26 unique nonamer peptides

In [ ]:
len(set(leader_peps.keys()))

### Exon 1 Seqs (Tables 1 + 4)

In [ ]:
order = ['AFA', 'API', 'CAU', 'HIS', 'NAM', 'MLT', 'UNK']

leaders_df = pd.DataFrame([[Sequence('').translate(pep.name), 
                            Sequence(pep.name, 
                                     ref_sequence=Sequence("GTCATGGCGCCCCGAACCGTCCTCCTG")) \
                                    .formatted()] +
                           [race in pep.broad_races and pep.broad_races[race]
               or 0
               for race in order] +
              [pep.count] for pep in exon1_seqs.values()],
            columns=['Peptide', 'Sequence'] + order + ['Total'])
leader_cols = leaders_df.columns
leader_cols = leader_cols[leader_cols != 'Peptide']
for col in leader_cols:
    leaders_df[col] = leaders_df[col] \
        .apply(lambda x : x or 0)
leaders_df = leaders_df[leaders_df['Total'] > 0] \
    .sort_values(by="Total", ascending=False)

# # Add percentages
# for broad_race in order:
#     leaders_df[broad_race] = leaders_df[broad_race] / leaders_df["Total"]
print(total)
print(leaders_df)
leaders_df.to_csv('results/tables_1&4_leader_peptide_seqs.csv')

# Allele Family

In [ ]:
allele_families = {}
alleles = {}
num_alleles = 0
for subject in subjects.values():
    for allele_index, allele_list in subject.alleles.items():
        num_alleles += 1
        allele_list.id = subject.id
        family = allele_list.allele_family
        if not family:
            print(allele_list)
        if family not in allele_families:
            allele_families[family] = AlleleFamily(family)
        if allele_list.leader_peptide or True:
            allele_families[family].add_allele(allele_list, subject.broad_race)

            for allele in allele_list.allele_list.split('/'):
                if allele not in alleles:
                    alleles[allele] = Allele(allele)
                alleles[allele].add_allele(allele_list, subject.broad_race)
        else:
            print(allele_list)
# print(alleles)

## Allele Family Table (Table 2)

In [ ]:
for i, allele_family_name in enumerate(sorted(allele_families.keys())):
    allele_family = allele_families[allele_family_name]
    data = {}
    total_count = sum([leader.count for leader in allele_family.leaders.values()])
    for leader in allele_family.leaders.values():
        leaders = allele_family.leaders[leader.name]
        leader_name = leader.name
        if leader.name not in ['M', 'T']:
            leader_name = 'Other'
        data['Allele Family'] = [allele_family.name]
        data[leader_name] = ['Total']
        data[leader_name + ' counts'] = [leaders.count]
    family_df = pd.DataFrame(data)
    if i == 0:
        families_df = family_df
    else:
        families_df = families_df.append(family_df)

# Apply totals row-wise
def get_cols(df, substring):
    return list(df.columns[df.columns.str.contains(substring)])
count_cols = get_cols(families_df, 'counts')
families_df.fillna(0, inplace=True)
families_df['Total'] = families_df.apply(lambda row: sum([row[count_col] for count_col in count_cols]), axis=1)

# Apply totals column-wise
totals = {'Allele Family' : ['Total'],
           'M' : [''],
           'M counts' : [families_df['M counts'].sum()],
           'T' : [''],
           'T counts' : [families_df['T counts'].sum()],
           'Other counts' : [families_df['Other counts'].sum()],
           'Total' : [families_df['Total'].sum()]
         }
total_df = pd.DataFrame(totals)

# Apply percentages
def label_perc(row, header):
    return "%.0f (%.2f%%)" % (row[header], (row[header] / row['Total'] * 100))
for count_col in count_cols:
    families_df[count_col] = families_df.apply(lambda row: label_perc(row, count_col), axis=1)

# Append totals and drop columns, empty values; export
families_df = families_df.append(total_df) \
    .drop(columns=["M", "T", "Other"]) #"M percentages", "T percentages", "Other percentages"])
families_df.replace('0 (0.00%)', '-', inplace=True)
families_df.to_csv('results/table_2_allele_families_leaders.csv', index=False)

### Minor :XX Alleles

#### B07 family

In [ ]:
allele_t = allele_families['07'].leaders['T'].alleles[0]

In [ ]:
allele_t.allele_list

In [ ]:
allele_families['07'].leaders['T']

In [ ]:
hla_db.align_allele_family('B*07:02:01:01', str(allele_t.whole_NTs), 
                           diff_limit=10, verbose=True, verbosity=2)

In [ ]:
hla_db.alleles['B*07:390'].gene_features.seq.alignment

In [ ]:
hla_db.alleles['B*07:02:01:01'].gene_features.seq

- The T minor allele closely matches B\*07:02:01:01. 5 differences on exon 1 (-G- - -C- --A) and 3' UTR (C---------C).
- Matches closer with B\*07:390. Same protein (2 missense substitutions compared to B\*07:02:01:01) but has 1 synonymous mutation within coding region (G->A) and three mutations outside coding region.

In [ ]:
hla_db.align_allele_family('B*07:02:01:02', str(allele_t.whole_NTs), limits=[255, 302],
                           verbose=True, verbosity=1)

In [ ]:
allele_v = allele_families['07'].leaders['V'].alleles[0]

In [ ]:
allele_families['07'].leaders['V']

In [ ]:
hla_db.align('B*07:371', str(allele_v.whole_NTs), verbose=True, verbosity=2)

V variant is B\*07:371

#### B08 family

M leader majority. One instance of T minor leader

In [ ]:
alleles['HLA-B*08:XX'].leaders

##### Major Leaders (M)

In [ ]:
major_08XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*08:XX'].leaders['M'].alleles]

In [ ]:
print("Length: %s. Unique length: %s" % (len(major_08XXs), len(set(major_08XXs))))

49 sequences. 38 unique sequences

In [ ]:
aligned_allele_sets = []
for i, major_08XX in enumerate(set(major_08XXs)):
    print(i)
    aligned_allele_set = hla_db.align_allele_family('08', major_08XX)
    aligned_allele_sets.append(aligned_allele_set)

##### Minor Leader (T)

In [ ]:
alleles['HLA-B*08:XX'].leaders['T']

Minor leader is a B\*08:207 variant with one substition

In [ ]:
minor_08xx = str(alleles['HLA-B*08:XX'].leaders['T'].alleles[0].whole_NTs)
# hla_db.align_allele_family('B*08', minor_08xx, diff_limit=10, verbose=True)

In [ ]:
hla_db.align('B*08:207', minor_08xx, limits=[36, 102], verbose=True, verbosity=1)

#### Subject Lookup

In [ ]:
alleles['HLA-B*08:XX'].leaders['T'].alleles[0].id

In HML, listed as 'B\*08:01:01+B\*08:XX'. On CORE, reported as 'B\*08:01'+'B\*08:01:01G'.

#### B13

T leader majority. One instance of M minor leader

In [ ]:
alleles['HLA-B*13:XX'].leaders

##### Major Leaders (T)

In [ ]:
major_13XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*13:XX'].leaders['T'].alleles]

In [ ]:
print("Length: %s. Unique length: %s" % (len(major_13XXs), len(set(major_13XXs))))

21 sequences. 13 unique sequences

##### Minor Leader (M)

Minor leader is found to be B\*13:117

In [ ]:
alleles['HLA-B*13:XX'].leaders['M']

In [ ]:
minor_13xx = str(alleles['HLA-B*13:XX'].leaders['M'].alleles[0].whole_NTs)
hla_db.align_allele_family('13:117', minor_13xx, verbose=True, diff_limit=2, verbosity=1)

##### Subject Lookup

In [ ]:
alleles['HLA-B*13:XX'].leaders['M'].alleles[0].id

Reported as B\*13:XX+B\*44:03:02

#### B15

In [ ]:
print(allele_families['15'].leaders['M'].alleles[0].allele_list)
print(allele_families['15'].leaders['M'].alleles[1].allele_list)

#### B18

T leader majority. One instance of M minor leader

In [ ]:
alleles['HLA-B*18:XX'].leaders

In [ ]:
allele_families['18']

##### Minor Leader (M)

There are three counts of an identical minor leader sequence.

In [ ]:
minor_XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*18:XX'].leaders['M'].alleles]
len(set(minor_XXs))

Difference of two nucleotides (one missense leader type mutation and one noncoding mutation ) for all of the following:
- 'B\*18:01:01:02'
- 'B\*18:01:01:04'
- 'B\*18:01:01:05'
- 'B\*18:01:01:16'
- 'B\*18:01:01:17'
- 'B\*18:01:01:20'
- 'B\*18:01:01:22'
- 'B\*18:01:01:47'
- 'B\*18:01:01:52'

In [ ]:
limits = [0,70]
aligned_alleles = hla_db.align_allele_family('18:01:01', minor_XXs[0], diff_limit=3,
                                             verbose=True, verbosity=1)

##### Subject Lookup

In [ ]:
[(allele.id, allele.source) for allele in alleles['HLA-B*18:XX'].leaders['M'].alleles]

#### B27

In [ ]:
allele1 = allele_families['27'].leaders['M'].alleles[0]
allele1.allele_list
allele2 = subjects[allele.id].alleles['allele1']
allele2.allele_list

In [ ]:
str(allele1.whole_NTs) == str(allele2.whole_NTs)

#### B35

T leader majority. One instance of M minor leader

In [ ]:
alleles['HLA-B*35:XX'].leaders

In [ ]:
allele_families['35']

##### Minor Leader (M)

There are 3 unique sequences out of 26 total

In [ ]:
minor_XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*35:XX'].leaders['M'].alleles]
print("Length: %s. Unique length: %s" % (len(minor_XXs), len(set(minor_XXs))))

In [ ]:
minor_XXs_df = pd.DataFrame(pd.Series(minor_XXs).value_counts())
minor_XXs_df.reset_index(inplace=True)
minor_XXs_df.columns = ['Sequence', 'Counts']
minor_XXs_df

All minor B\*35:XX sequences align to the following by 3 of the same substitutions:
- B\*35:01:01:01
- B\*35:01:01:02
- B\*35:01:01:04
- B\*35:01:01:05
- B\*35:01:01:06
- B\*35:01:01:13
- B\*35:01:01:14
- B\*35:01:01:16
- B\*35:01:01:18
- B\*35:01:01:19
- B\*35:01:01:49
- B\*35:01:01:51

Check this:
- B\*35:447 (4 differences but it says 3)


In [ ]:
aligned_allele_sets = []
limits = [0, 47]
for i, row in minor_XXs_df.iterrows():
    if i == 1:
        break
    print(i, row['Counts'])
    aligned_allele_set = hla_db.align_allele_family('', row['Sequence'], limits=limits, diff_limit=3, verbose=True, verbosity=0)
    aligned_allele_sets.append(aligned_allele_set)

##### Subject Lookup

In [ ]:
[(allele.id, allele.source) for allele in alleles['HLA-B*35:XX'].leaders['M'].alleles]

Reported as B\*35:XX (25), B\*35:01:01G (1)

#### B38

In [ ]:
allele1 = allele_families['38'].leaders['T'].alleles[0]
print(allele1.allele_list)
allele2 = subjects[allele1.id].alleles['allele1']
# hla_db.align_allele_family(allele.allele_list.replace('HLA-',''),
#                            str(allele.whole_NTs), verbose=True, verbosity=0)
print(allele2.allele_list)
allele1.whole_NTs == allele2.whole_NTs

#### B39

In [ ]:
allele.id

In [ ]:
allele = allele_families['39'].leaders['T'].alleles[0]
# allele2 = subjects[allele1.id].alleles['allele1']
hla_db.align_allele_family('B*39',
                           str(allele.whole_NTs), verbose=True, verbosity=1)

#### B40

T leader majority. Two instances of M minor leader

In [ ]:
alleles['HLA-B*40:XX'].leaders

##### Minor Leader (M)

There are two counts of an identical minor leader sequence.

In [ ]:
minor_XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*40:XX'].leaders['M'].alleles]
len(set(minor_XXs))

Found to be B\*40:416

In [ ]:
limits = [0,108]
limits = None
aligned_alleles = hla_db.align_allele_family('40', minor_XXs[0], diff_limit=1, limits=limits, verbose=True, verbosity=0)

##### Subject Lookup

In [ ]:
[(allele.id, allele.source) for allele in alleles['HLA-B*40:XX'].leaders['M'].alleles]

In [ ]:
allele_families['51'].leaders['M'].alleles[0].__dict__

In [ ]:
str(allele.whole_NTs)

In [ ]:
hla_db.get_processed_ref_seq("B*51:01:01:01", str(allele.whole_NTs))

In [ ]:
allele = allele_families['51'].leaders['M'].alleles[0]
print(allele.allele_list)
hla_db.align_allele_family("B*51:01:01:01", 
                           str(allele.whole_NTs), diff_limit = 5, verbose=True, verbosity=1)

#### B55

T leader majority. One instances of 'R' minor leader

In [ ]:
alleles['HLA-B*55:XX'].leaders

In [ ]:
allele_families['55']

##### Minor Leader (R)

There are two counts of an identical minor leader sequence.

In [ ]:
minor_XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*55:XX'].leaders['R'].alleles]
len(set(minor_XXs))

Found to be B\*55:91 (R)

In [ ]:
limits = [0,108]
limits = None
aligned_alleles = hla_db.align_allele_family('55', minor_XXs[0], diff_limit=1, limits=limits, verbose=True, verbosity=0)

##### Subject Lookup

In [ ]:
[(allele.id, allele.source) for allele in alleles['HLA-B*55:XX'].leaders['R'].alleles]

#### B\*58:01

T leader majority. One instances of 'M' minor leader.

In [ ]:
alleles['HLA-B*58:01'].leaders

##### Minor Leader (R)

There are two counts of an identical minor leader sequence.

In [ ]:
minor_XXs = [str(allele.whole_NTs) for allele in alleles['HLA-B*58:01'].leaders['M'].alleles]
len(set(minor_XXs))

Found to be B\*55:91 (R)

In [ ]:
limits = [0,47]
# limits = None
aligned_alleles = hla_db.align_allele_family('58', minor_XXs[0], diff_limit=3, limits=limits, verbose=True, verbosity=1)

##### Subject Lookup

In [ ]:
[(allele.id, allele.source) for allele in alleles['HLA-B*58:01'].leaders['M'].alleles]

## Populations

In [ ]:
pd.Series([subject.broad_race for subject in subjects.values()]) \
    .value_counts(normalize=False)

In [ ]:
pd.Series([subject.broad_race for subject in subjects.values()]) \
    .value_counts(normalize=True)